In [ ]:
"""数据库操作

SQLite是一种嵌入式数据库，它的数据库就是一个文件
SQLite由C语言写成，体积很小，所以经常被集成到各种应用程序中，甚至移动APP中
Python内置了SQLite及相关驱动
MySQL的SQL占位符是%s
Python的DB-API定义对不同的数据库是通用的

操作数据库的基本步骤：创建连接 -> 获取游标 -> 执行SQL -> 关闭游标 -> 提交事务（非查询操作） -> 关闭连接
使用游标Cursor对象执行insert update delete语句时，执行结果由rowcount返回影响的行数
使用游标Cursor对象执行select语句时，通过fetchall()可以获取结果集
结果集是一个list，每个元素都是一个tuple，对应一行记录
执行INSERT等操作后要调用commit()提交事务

ORM(Object-Relational Mapping)对象关系映射
ORM就是把数据库表的行与相应的对象建立关联，把对表的操作转换为对对象的操作
常用ORM框架：SQLAlchemy
"""

In [10]:
"""SQLite基本操作"""

import sqlite3

# 建立数据库连接
# 如果连接的数据库文件不存在，则自动创建
conn = sqlite3.connect('./data/test.db')

# 获取游标
cur = conn.cursor()

# 执行SQL，创建user表
# cur.execute('DROP TABLE user')
# cur.execute('CREATE TABLE user (id varchar(20) primary key, name varchar(20))')

# 执行SQL，插入一条记录
# cur.execute('INSERT INTO user (id, name) VALUES ("1", "Neo")')

# 获取插入行数
# print('成功插入了{}条数据'.format(cur.rowcount))

# 执行查询
cur.execute('SELECT * FROM user WHERE id=?', ('1',))
res = cur.fetchall()
print('查询结果：', res)

# 关闭游标
cur.close()

# 提交事务
# conn.commit()

# 关闭连接
conn.close()

查询结果： [('1', 'Neo')]


In [13]:
"""MySQL基本操作"""

import mysql.connector

# 建立数据库连接
conn = mysql.connector.connect(user='root', password='123456', database='sys')

# 获取游标
cur = conn.cursor()

# 执行查询
cur.execute('SELECT * FROM sys_config WHERE 1=1')
res = cur.fetchall()

print('查询结果：')
for r in res:
    print(r[0], ': ', r[1])
    
# 关闭游标
cur.close()

# 关闭连接
conn.close()

查询结果：
diagnostics.allow_i_s_tables :  OFF
diagnostics.include_raw :  OFF
ps_thread_trx_info.max_length :  65535
statement_performance_analyzer.limit :  100
statement_performance_analyzer.view :  None
statement_truncate_len :  64


In [16]:
"""对象关系映射ORM"""

from sqlalchemy import Column, String, create_engine, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

# 创建对象基类
Base = declarative_base()

# 自定义对象 - 映射数据表
class User(Base):
    # 表名
    __tablename__ = 'user'
    
    # 表结构
    id = Column(String(20), primary_key=True)
    name = Column(String(20))
    
    # 表关联
    # 一对多
    # articles = relationship('Article')
    # 多对一
    # city_id = Column(String(20), ForeignKey('city_id'))
    
# 初始化数据库连接
# 数据库类型+数据库驱动名称://用户名:口令@机器地址:端口号/数据库名
engine = create_engine('sqlite:///test.db')

# 创建DBSession
DBSession = sessionmaker(bind=engine)

# 创建session对象
session = DBSession()

# 创建User对象
# new_user = User(id='1', name='Neo')
# 添加到session
# session.add(new_user)
# 提交保存到数据库
# session.commit()

# 执行查询
query_user = session.query(User).filter(User.id == '1').one()
print('type: ', type(query_user))
print('name: ', query_user.name)

# 关闭session
session.close()